In [1]:
import requests
import json
import time
import pandas as pd
from datetime import datetime
import numpy as np
from ratelimit import limits

<font size="5">ZENDESK</font>

In [2]:
# Set the request parameters TICKET_IDs by DATE

zendesk_raw_data = []

def zendesk_limit():
    url = 'https://YOUR_ZENDESK_DOMAIN.zendesk.com/api/v2/search.json?query=type%3Aticket+created%3A2020-07-01'
    user = 'YOUR_EMAIL' + '/token' #you don't need to have your specific "token", it's just a value needed here as it was placed
    pwd = 'YOUR_PASSWORD'
    response = requests.get(url, auth=(user, pwd))
    data = response.json()
    zendesk_raw_data.append(data)
    while zendesk_raw_data[-1].get('next_page') != None: #here we check if the last page of our final list contains value for "next_page", if it is - we continue to request data by changing the value of requested url
        url_next_page = zendesk_raw_data[-1].get('next_page')
        response_next = requests.get(url_next_page, auth=(user, pwd))
        data_next = response_next.json()
        zendesk_raw_data.append(data_next)
    print("Download complete. Total number of pages: " + str(len(zendesk_raw_data)))

In [3]:
zendesk_limit()

Download complete. Total number of pages: 1


In [4]:
#Removes "result" value from each list

zendesk_list_data = []

for i in range(len(zendesk_raw_data)):
    zendesk_list_data.append(list(zendesk_raw_data[i].values())[0])

In [ ]:
#Creating dataframe for individual tickets + Date

is_public = []
ticket_id = []
created_at = []
submitter_id = []

for i in range(len(zendesk_list_data)):
    for z in range(len(zendesk_list_data[i])):
        ticket_id.append(zendesk_list_data[i][z].get('id'))
        created_at.append(zendesk_list_data[i][z].get('created_at'))
        is_public.append(zendesk_list_data[i][z].get('is_public'))
        submitter_id.append(zendesk_list_data[i][z].get('submitter_id'))
        
column_names_2 = ["ticket_id", "created_at", "submitter_id", "is_public", "date"]

zendesk_ticket_by_date = pd.DataFrame(columns = column_names_2)

zendesk_ticket_by_date["ticket_id"] = ticket_id
zendesk_ticket_by_date["created_at"] = created_at
zendesk_ticket_by_date["submitter_id"] = submitter_id
zendesk_ticket_by_date["is_public"] = is_public

In [ ]:
#Convert date columns to a DateTime format

zendesk_ticket_by_date['created_at'] = pd.to_datetime(zendesk_ticket_by_date['created_at'])

In [ ]:
#Creating Date column from API data in a clean format, like: 2020-07-09

def date_convertion_zen(row):
    try:
        new_date = datetime.strftime(row, '%Y-%m-%d')
        return new_date
    except ValueError:
        return None

zendesk_ticket_by_date['date'] = zendesk_ticket_by_date['created_at'].apply(lambda row: date_convertion_zen(row))

#It's not required too much, it was done to make sure that data in dataframe has the same format as column data type in SQL database

zendesk_ticket_by_date['date'] = pd.to_datetime(zendesk_ticket_by_date['date'])

In [ ]:
#Sort df by Ticket_id column

zendesk_ticket_by_date = zendesk_ticket_by_date.sort_values(by=['ticket_id'])

In [ ]:
# Important step - we are creating list of all tickets ID for selected date. 
# Next we'll getting new data from Zendesk for each ID from this list.

tickets_id_list = zendesk_ticket_by_date['ticket_id'].tolist()

In [ ]:
# Just check how many records we will have for selected date

print(len(tickets_id_list))

In [ ]:
#Get metric_data for each ticket by ID from previous step
#Set limit for API requests (in this case 300 requests per 1 min) 

zendesk_tickets_metric_raw = []

One_minute = 60

@limits(calls=300, period = One_minute)
def zendesk_ticket_metric():
    user = 'YOUR_EMAIL' + '/token'
    pwd = 'YOUR_PASSWORD'
    for d in tickets_id_list:
        url = 'https://YOUR_ZENDESK_DOMAIN.zendesk.com/api/v2/tickets/' + str(d) + '/metrics.json'
        response = requests.get(url, auth=(user, pwd))
        data_ticket = response.json()
        zendesk_tickets_metric_raw.append(data_ticket)
    print("Download complete. Total number of pages: " + str(len(zendesk_tickets_metric_raw)))

In [ ]:
zendesk_ticket_metric()

In [ ]:
#Removes "result" value from each list

zendesk_ticket_metric_clean = []

for i in range(len(zendesk_tickets_metric_raw)):
    zendesk_ticket_metric_clean.append(list(zendesk_tickets_metric_raw[i].values())[0])

In [ ]:
#Creating dataframe for individual ticket + Time_replies

metric_id = []
ticket_id = []
created_at = []
solved_at = []
reply_time_in_minutes = []
first_resolution_time_in_minutes = []
requester_wait_time_in_minutes = []

for i in range(len(zendesk_ticket_metric_clean)):
    metric_id.append(zendesk_ticket_metric_clean[i].get('id'))
    ticket_id.append(zendesk_ticket_metric_clean[i].get('ticket_id'))
    created_at.append(zendesk_ticket_metric_clean[i].get('created_at'))
    solved_at.append(zendesk_ticket_metric_clean[i].get('solved_at'))
    reply_time_in_minutes.append(zendesk_ticket_metric_clean[i].get('reply_time_in_minutes').get('calendar'))
    first_resolution_time_in_minutes.append(zendesk_ticket_metric_clean[i].get('first_resolution_time_in_minutes').get('calendar'))
    requester_wait_time_in_minutes.append(zendesk_ticket_metric_clean[i].get('requester_wait_time_in_minutes').get('calendar'))  


column_names_3 = ["metric_id", "ticket_id", "created_at", "solved_at", "reply_time_in_minutes", "first_resolution_time_in_minutes", "requester_wait_time_in_minutes"]

zendesk_data_ticket_metric = pd.DataFrame(columns = column_names_3)

zendesk_data_ticket_metric["metric_id"] = metric_id
zendesk_data_ticket_metric["ticket_id"] = ticket_id
zendesk_data_ticket_metric["created_at"] = created_at
zendesk_data_ticket_metric["solved_at"] = solved_at
zendesk_data_ticket_metric["reply_time_in_minutes"] = reply_time_in_minutes
zendesk_data_ticket_metric["first_resolution_time_in_minutes"] = first_resolution_time_in_minutes
zendesk_data_ticket_metric["requester_wait_time_in_minutes"] = requester_wait_time_in_minutes

In [ ]:
#Sort df_metric by Ticket_id column

zendesk_data_ticket_metric = zendesk_data_ticket_metric.sort_values(by=['ticket_id'])

In [ ]:
#Merging two dataframes: Tickets by selected Date + Tickets metric

final_zendesk = pd.merge(zendesk_ticket_by_date, zendesk_data_ticket_metric, on='ticket_id')

In [ ]:
#Some final dataframe cleaning

final_zendesk.drop(columns=["created_at_x"], inplace=True)
final_zendesk.rename(columns={'created_at_y':'created_at'}, inplace=True)
final_zendesk["reply_time_in_minutes"] = final_zendesk["reply_time_in_minutes"].fillna(0)
final_zendesk["reply_time_in_minutes"] = final_zendesk["reply_time_in_minutes"].astype('int64')
final_zendesk['date'] = pd.to_datetime(final_zendesk['date'])
final_zendesk['created_at'] = pd.to_datetime(final_zendesk['created_at'])
final_zendesk['solved_at'] = pd.to_datetime(final_zendesk['solved_at'])

In [ ]:
#Reshaping columns in needed order

final_df_columns = ['ticket_id', 'metric_id', 'submitter_id', 'is_public', 'reply_time_in_minutes', 'first_resolution_time_in_minutes',
                    'requester_wait_time_in_minutes', 'created_at', 'solved_at', 'date']
final_zendesk = final_zendesk[final_df_columns]

In [ ]:
#Save results in CSV format

final_zendesk.to_csv("Ticket_list_metric.csv", index=False)